# COVID19 Notebook
website: `https://github.com/CSSEGISandData/COVID-19`

```bash
cd ~/cisc_525
hdfs dfs -copyFromLocal COVID-19 /user/student

jupyter notebook
```

In [32]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
import subprocess
from pyspark.sql.functions import col, max as max_
import datetime

In [33]:
spark = SparkSession.builder.appName("covid19-app").config("spark.config.option", "value").getOrCreate()
scfg = SparkConf().setAppName('covid19-app')


In [34]:
day = 'hdfs://localhost:9000/user/student/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us/05-15-2020.csv'
time_series = 'hdfs://localhost:9000/user/student/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'

In [35]:
ts_df = spark.read.option('header', 'true').csv(time_series)
day_df = spark.read.option('header', 'true').csv(day)

## RDD Section

In [36]:
# Converting from a dataframe df to a resilient distributed data rdd
ts_rdd = ts_df.rdd

In [38]:
# Data Column number

# UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,
# 1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,
# 2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,
# 2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,
# 2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,
# 3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,
# 3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,
# 3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,
# 4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,
# 4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,
# 5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,
# 5/15/20

TS_COLUMNS = ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
              'Country_Region', 'Lat', 'Long_', 'Combined_Key']

TS_DATE_START_COLUMN = 11

ts_first_row = ts_rdd.first()
ts_first_row


Row(UID='16', iso2='AS', iso3='ASM', code3='16', FIPS='60.0', Admin2=None, Province_State='American Samoa', Country_Region='US', Lat='-14.271', Long_='-170.132', Combined_Key='American Samoa, US', 1/22/20='0', 1/23/20='0', 1/24/20='0', 1/25/20='0', 1/26/20='0', 1/27/20='0', 1/28/20='0', 1/29/20='0', 1/30/20='0', 1/31/20='0', 2/1/20='0', 2/2/20='0', 2/3/20='0', 2/4/20='0', 2/5/20='0', 2/6/20='0', 2/7/20='0', 2/8/20='0', 2/9/20='0', 2/10/20='0', 2/11/20='0', 2/12/20='0', 2/13/20='0', 2/14/20='0', 2/15/20='0', 2/16/20='0', 2/17/20='0', 2/18/20='0', 2/19/20='0', 2/20/20='0', 2/21/20='0', 2/22/20='0', 2/23/20='0', 2/24/20='0', 2/25/20='0', 2/26/20='0', 2/27/20='0', 2/28/20='0', 2/29/20='0', 3/1/20='0', 3/2/20='0', 3/3/20='0', 3/4/20='0', 3/5/20='0', 3/6/20='0', 3/7/20='0', 3/8/20='0', 3/9/20='0', 3/10/20='0', 3/11/20='0', 3/12/20='0', 3/13/20='0', 3/14/20='0', 3/15/20='0', 3/16/20='0', 3/17/20='0', 3/18/20='0', 3/19/20='0', 3/20/20='0', 3/21/20='0', 3/22/20='0', 3/23/20='0', 3/24/20='0', 3/

In [39]:
# Iterating through the list of values of the first row
for val in ts_first_row:
    print(val)


16
AS
ASM
16
60.0
None
American Samoa
US
-14.271
-170.132
American Samoa, US
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [40]:
# Iterate through the list of values of first row using column name with ROW data type
for key in TS_COLUMNS:
    print(key, '=', ts_first_row[key])
    
# ts_by_dates = ts_first[TS_DATE_START_COLUMN:]
# for ts_by_date in ts_by_dates:
#     print(ts_by_date)

UID = 16
iso2 = AS
iso3 = ASM
code3 = 16
FIPS = 60.0
Admin2 = None
Province_State = American Samoa
Country_Region = US
Lat = -14.271
Long_ = -170.132
Combined_Key = American Samoa, US


In [41]:
# Something about date and time
start_date = datetime.date(2020, 1, 22)
print (start_date.strftime('%m/%d/%y'))
start_date += datetime.timedelta(days=1)
cur_date_str = '{}/{}/{}'.format(start_date.month, start_date.day, start_date.year-2000)
print (start_date.strftime('%0m/%d/%y'))
print(cur_date_str)
print(cur_date_str == '1/23/20')

cur_date = datetime.date(2020, 1, 22)
DATE_COLUMNS = []
while True:
    cur_date += datetime.timedelta(days=1)
    cur_date_str = '{}/{}/{}'.format(cur_date.month, cur_date.day, cur_date.year-2000)
    print(cur_date_str)
    DATE_COLUMNS.append(cur_date_str)
    if cur_date_str == '6/7/20':
        break

for date_str in DATE_COLUMNS:
    print(date_str)

01/22/20
01/23/20
1/23/20
True
1/23/20
1/24/20
1/25/20
1/26/20
1/27/20
1/28/20
1/29/20
1/30/20
1/31/20
2/1/20
2/2/20
2/3/20
2/4/20
2/5/20
2/6/20
2/7/20
2/8/20
2/9/20
2/10/20
2/11/20
2/12/20
2/13/20
2/14/20
2/15/20
2/16/20
2/17/20
2/18/20
2/19/20
2/20/20
2/21/20
2/22/20
2/23/20
2/24/20
2/25/20
2/26/20
2/27/20
2/28/20
2/29/20
3/1/20
3/2/20
3/3/20
3/4/20
3/5/20
3/6/20
3/7/20
3/8/20
3/9/20
3/10/20
3/11/20
3/12/20
3/13/20
3/14/20
3/15/20
3/16/20
3/17/20
3/18/20
3/19/20
3/20/20
3/21/20
3/22/20
3/23/20
3/24/20
3/25/20
3/26/20
3/27/20
3/28/20
3/29/20
3/30/20
3/31/20
4/1/20
4/2/20
4/3/20
4/4/20
4/5/20
4/6/20
4/7/20
4/8/20
4/9/20
4/10/20
4/11/20
4/12/20
4/13/20
4/14/20
4/15/20
4/16/20
4/17/20
4/18/20
4/19/20
4/20/20
4/21/20
4/22/20
4/23/20
4/24/20
4/25/20
4/26/20
4/27/20
4/28/20
4/29/20
4/30/20
5/1/20
5/2/20
5/3/20
5/4/20
5/5/20
5/6/20
5/7/20
5/8/20
5/9/20
5/10/20
5/11/20
5/12/20
5/13/20
5/14/20
5/15/20
5/16/20
5/17/20
5/18/20
5/19/20
5/20/20
5/21/20
5/22/20
5/23/20
5/24/20
5/25/20
5/26/20
5/27/

In [42]:
# Printing out the content of the values by date columns. 
# Date columns are to extend over time.

start_date = datetime.date(2020, 1, 22)
ts_first = ts_rdd.first();
ts_by_dates = ts_first[TS_DATE_START_COLUMN:]

# for ts_by_date in ts_by_dates:
#     print(ts_by_date)
    
for date_str in DATE_COLUMNS:
    print(date_str, '=', ts_first[date_str])

1/23/20 = 0
1/24/20 = 0
1/25/20 = 0
1/26/20 = 0
1/27/20 = 0
1/28/20 = 0
1/29/20 = 0
1/30/20 = 0
1/31/20 = 0
2/1/20 = 0
2/2/20 = 0
2/3/20 = 0
2/4/20 = 0
2/5/20 = 0
2/6/20 = 0
2/7/20 = 0
2/8/20 = 0
2/9/20 = 0
2/10/20 = 0
2/11/20 = 0
2/12/20 = 0
2/13/20 = 0
2/14/20 = 0
2/15/20 = 0
2/16/20 = 0
2/17/20 = 0
2/18/20 = 0
2/19/20 = 0
2/20/20 = 0
2/21/20 = 0
2/22/20 = 0
2/23/20 = 0
2/24/20 = 0
2/25/20 = 0
2/26/20 = 0
2/27/20 = 0
2/28/20 = 0
2/29/20 = 0
3/1/20 = 0
3/2/20 = 0
3/3/20 = 0
3/4/20 = 0
3/5/20 = 0
3/6/20 = 0
3/7/20 = 0
3/8/20 = 0
3/9/20 = 0
3/10/20 = 0
3/11/20 = 0
3/12/20 = 0
3/13/20 = 0
3/14/20 = 0
3/15/20 = 0
3/16/20 = 0
3/17/20 = 0
3/18/20 = 0
3/19/20 = 0
3/20/20 = 0
3/21/20 = 0
3/22/20 = 0
3/23/20 = 0
3/24/20 = 0
3/25/20 = 0
3/26/20 = 0
3/27/20 = 0
3/28/20 = 0
3/29/20 = 0
3/30/20 = 0
3/31/20 = 0
4/1/20 = 0
4/2/20 = 0
4/3/20 = 0
4/4/20 = 0
4/5/20 = 0
4/6/20 = 0
4/7/20 = 0
4/8/20 = 0
4/9/20 = 0
4/10/20 = 0
4/11/20 = 0
4/12/20 = 0
4/13/20 = 0
4/14/20 = 0
4/15/20 = 0
4/16/20 = 0
4/17/20

### Group By Province or State


In [43]:
# When we group by provice or state, we get a list of nodes. each node
# consists of the key (name of the state) and a list of da rows for each 
# of the states.

ts_states = ts_rdd.groupBy(lambda x: x['Province_State'])
# dir(ts_states)
sorted_by_states = ts_states.sortByKey('Province_State')
# print(sorted_by_states)

sorted_by_states.collect()

for row in sorted_by_states.collect():
    print(row[0], len(row[1]))

Alabama 69
Alaska 31
American Samoa 1
Arizona 17
Arkansas 77
California 60
Colorado 66
Connecticut 10
Delaware 5
Diamond Princess 1
District of Columbia 3
Florida 69
Georgia 161
Grand Princess 1
Guam 1
Hawaii 7
Idaho 46
Illinois 104
Indiana 94
Iowa 101
Kansas 107
Kentucky 122
Louisiana 66
Maine 18
Maryland 26
Massachusetts 17
Michigan 87
Minnesota 89
Mississippi 84
Missouri 118
Montana 58
Nebraska 95
Nevada 19
New Hampshire 12
New Jersey 23
New Mexico 35
New York 64
North Carolina 102
North Dakota 55
Northern Mariana Islands 1
Ohio 90
Oklahoma 79
Oregon 38
Pennsylvania 69
Puerto Rico 1
Rhode Island 7
South Carolina 48
South Dakota 68
Tennessee 97
Texas 256
Utah 37
Vermont 16
Virgin Islands 1
Virginia 135
Washington 41
West Virginia 57
Wisconsin 74
Wyoming 25


In [44]:

print(len(ts_states.collect()))
for state in sorted(ts_states.collect()):
    print(state[0], len(state[1]))
    for item in state[1]:
        print ('\t', item['Admin2'])

58
Alabama 69
	 Autauga
	 Baldwin
	 Barbour
	 Bibb
	 Blount
	 Bullock
	 Butler
	 Calhoun
	 Chambers
	 Cherokee
	 Chilton
	 Choctaw
	 Clarke
	 Clay
	 Cleburne
	 Coffee
	 Colbert
	 Conecuh
	 Coosa
	 Covington
	 Crenshaw
	 Cullman
	 Dale
	 Dallas
	 DeKalb
	 Elmore
	 Escambia
	 Etowah
	 Fayette
	 Franklin
	 Geneva
	 Greene
	 Hale
	 Henry
	 Houston
	 Jackson
	 Jefferson
	 Lamar
	 Lauderdale
	 Lawrence
	 Lee
	 Limestone
	 Lowndes
	 Macon
	 Madison
	 Marengo
	 Marion
	 Marshall
	 Mobile
	 Monroe
	 Montgomery
	 Morgan
	 Perry
	 Pickens
	 Pike
	 Randolph
	 Russell
	 St. Clair
	 Shelby
	 Sumter
	 Talladega
	 Tallapoosa
	 Tuscaloosa
	 Walker
	 Washington
	 Wilcox
	 Winston
	 Out of AL
	 Unassigned
Alaska 31
	 Aleutians East
	 Aleutians West
	 Anchorage
	 Bethel
	 Bristol Bay
	 Denali
	 Dillingham
	 Fairbanks North Star
	 Haines
	 Hoonah-Angoon
	 Juneau
	 Kenai Peninsula
	 Ketchikan Gateway
	 Kodiak Island
	 Kusilvak
	 Lake and Peninsula
	 Matanuska-Susitna
	 Nome
	 North Slope
	 Northwest Arctic


	 McPherson
	 Marion
	 Marshall
	 Meade
	 Miami
	 Mitchell
	 Montgomery
	 Morris
	 Morton
	 Nemaha
	 Neosho
	 Ness
	 Norton
	 Osage
	 Osborne
	 Ottawa
	 Pawnee
	 Phillips
	 Pottawatomie
	 Pratt
	 Rawlins
	 Reno
	 Republic
	 Rice
	 Riley
	 Rooks
	 Rush
	 Russell
	 Saline
	 Scott
	 Sedgwick
	 Seward
	 Shawnee
	 Sheridan
	 Sherman
	 Smith
	 Stafford
	 Stanton
	 Stevens
	 Sumner
	 Thomas
	 Trego
	 Wabaunsee
	 Wallace
	 Washington
	 Wichita
	 Wilson
	 Woodson
	 Wyandotte
	 Out of KS
	 Unassigned
Kentucky 122
	 Adair
	 Allen
	 Anderson
	 Ballard
	 Barren
	 Bath
	 Bell
	 Boone
	 Bourbon
	 Boyd
	 Boyle
	 Bracken
	 Breathitt
	 Breckinridge
	 Bullitt
	 Butler
	 Caldwell
	 Calloway
	 Campbell
	 Carlisle
	 Carroll
	 Carter
	 Casey
	 Christian
	 Clark
	 Clay
	 Clinton
	 Crittenden
	 Cumberland
	 Daviess
	 Edmonson
	 Elliott
	 Estill
	 Fayette
	 Fleming
	 Floyd
	 Franklin
	 Fulton
	 Gallatin
	 Garrard
	 Grant
	 Graves
	 Grayson
	 Green
	 Greenup
	 Hancock
	 Hardin
	 Harlan
	 Harrison
	 Hart
	 Hender

	 Travis
	 Trinity
	 Tyler
	 Upshur
	 Upton
	 Uvalde
	 Val Verde
	 Van Zandt
	 Victoria
	 Walker
	 Waller
	 Ward
	 Washington
	 Webb
	 Wharton
	 Wheeler
	 Wichita
	 Wilbarger
	 Willacy
	 Williamson
	 Wilson
	 Winkler
	 Wise
	 Wood
	 Yoakum
	 Young
	 Zapata
	 Zavala
	 Out of TX
	 Unassigned
Utah 37
	 Beaver
	 Box Elder
	 Cache
	 Carbon
	 Daggett
	 Davis
	 Duchesne
	 Emery
	 Garfield
	 Grand
	 Iron
	 Juab
	 Kane
	 Millard
	 Morgan
	 Piute
	 Rich
	 Salt Lake
	 San Juan
	 Sanpete
	 Sevier
	 Summit
	 Tooele
	 Uintah
	 Utah
	 Wasatch
	 Washington
	 Wayne
	 Weber
	 Out of UT
	 Unassigned
	 Bear River
	 Central Utah
	 Southeast Utah
	 Southwest Utah
	 TriCounty
	 Weber-Morgan
Vermont 16
	 Addison
	 Bennington
	 Caledonia
	 Chittenden
	 Essex
	 Franklin
	 Grand Isle
	 Lamoille
	 Orange
	 Orleans
	 Rutland
	 Washington
	 Windham
	 Windsor
	 Out of VT
	 Unassigned
Virgin Islands 1
	 None
Virginia 135
	 Accomack
	 Albemarle
	 Alleghany
	 Amelia
	 Amherst
	 Appomattox
	 Arlington
	 Augusta
	 Bath
	

In [45]:
# ts_filtered = ts_rdd.filter(lambda x: x['Admin2'] != None)
ts_admin2s = ts_rdd.groupBy(lambda x: x['Admin2'])
# ts_states
ts_admin2s = ts_admin2s.filter(lambda x: x[0] != None)
# print(ts_admin2s.collect())
print(len(ts_admin2s.collect()))
for admin2 in sorted(ts_admin2s.collect()):
    print(admin2[0], len(admin2[1]))


1901
Abbeville 1
Acadia 1
Accomack 1
Ada 1
Adair 4
Adams 12
Addison 1
Aiken 1
Aitkin 1
Alachua 1
Alamance 1
Alameda 1
Alamosa 1
Albany 2
Albemarle 1
Alcona 1
Alcorn 1
Aleutians East 1
Aleutians West 1
Alexander 2
Alexandria 1
Alfalfa 1
Alger 1
Allamakee 1
Allegan 1
Allegany 2
Alleghany 2
Allegheny 1
Allen 5
Allendale 1
Alpena 1
Alpine 1
Amador 1
Amelia 1
Amherst 1
Amite 1
Anchorage 1
Anderson 5
Andrew 1
Andrews 1
Androscoggin 1
Angelina 1
Anne Arundel 1
Anoka 1
Anson 1
Antelope 1
Antrim 1
Apache 1
Appanoose 1
Appling 1
Appomattox 1
Aransas 1
Arapahoe 1
Archer 1
Archuleta 1
Arenac 1
Arkansas 1
Arlington 1
Armstrong 2
Aroostook 1
Arthur 1
Ascension 1
Ashe 1
Ashland 2
Ashley 1
Ashtabula 1
Asotin 1
Assumption 1
Atascosa 1
Atchison 2
Athens 1
Atkinson 1
Atlantic 1
Atoka 1
Attala 1
Audrain 1
Audubon 1
Auglaize 1
Augusta 1
Aurora 1
Austin 1
Autauga 1
Avery 1
Avoyelles 1
Baca 1
Bacon 1
Bailey 1
Baker 3
Baldwin 2
Ballard 1
Baltimore 1
Baltimore City 1
Bamberg 1
Bandera 1
Banks 1
Banner 1
Bannoc

Idaho 1
Imperial 1
Independence 1
Indian River 1
Indiana 1
Ingham 1
Inyo 1
Ionia 1
Iosco 1
Iowa 2
Iredell 1
Irion 1
Iron 4
Iroquois 1
Irwin 1
Isabella 1
Isanti 1
Island 1
Isle of Wight 1
Issaquena 1
Itasca 1
Itawamba 1
Izard 1
Jack 1
Jackson 24
James City 1
Jasper 8
Jay 1
Jeff Davis 2
Jefferson 26
Jefferson Davis 2
Jenkins 1
Jennings 1
Jerauld 1
Jerome 1
Jersey 1
Jessamine 1
Jewell 1
Jim Hogg 1
Jim Wells 1
Jo Daviess 1
Johnson 12
Johnston 2
Jones 6
Josephine 1
Juab 1
Judith Basin 1
Juneau 2
Juniata 1
Kalamazoo 1
Kalawao 1
Kalkaska 1
Kanabec 1
Kanawha 1
Kandiyohi 1
Kane 2
Kankakee 1
Kansas City 1
Karnes 1
Kauai 1
Kaufman 1
Kay 1
Kearney 1
Kearny 1
Keith 1
Kemper 1
Kenai Peninsula 1
Kendall 2
Kenedy 1
Kennebec 1
Kenosha 1
Kent 5
Kenton 1
Keokuk 1
Kern 1
Kerr 1
Kershaw 1
Ketchikan Gateway 1
Kewaunee 1
Keweenaw 1
Keya Paha 1
Kidder 1
Kimball 1
Kimble 1
King 2
King George 1
King William 1
King and Queen 1
Kingfisher 1
Kingman 1
Kings 2
Kingsbury 1
Kinney 1
Kiowa 3
Kit Carson 1
Kitsap 1
Kitt

## Dataframe

In [46]:
ts_df

DataFrame[UID: string, iso2: string, iso3: string, code3: string, FIPS: string, Admin2: string, Province_State: string, Country_Region: string, Lat: string, Long_: string, Combined_Key: string, 1/22/20: string, 1/23/20: string, 1/24/20: string, 1/25/20: string, 1/26/20: string, 1/27/20: string, 1/28/20: string, 1/29/20: string, 1/30/20: string, 1/31/20: string, 2/1/20: string, 2/2/20: string, 2/3/20: string, 2/4/20: string, 2/5/20: string, 2/6/20: string, 2/7/20: string, 2/8/20: string, 2/9/20: string, 2/10/20: string, 2/11/20: string, 2/12/20: string, 2/13/20: string, 2/14/20: string, 2/15/20: string, 2/16/20: string, 2/17/20: string, 2/18/20: string, 2/19/20: string, 2/20/20: string, 2/21/20: string, 2/22/20: string, 2/23/20: string, 2/24/20: string, 2/25/20: string, 2/26/20: string, 2/27/20: string, 2/28/20: string, 2/29/20: string, 3/1/20: string, 3/2/20: string, 3/3/20: string, 3/4/20: string, 3/5/20: string, 3/6/20: string, 3/7/20: string, 3/8/20: string, 3/9/20: string, 3/10/20:

In [47]:
dir(ts_df)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collectAsArrow',
 '_jcols',
 '_jdf',
 '_jmap',
 '_jseq',
 '_lazy_rdd',
 '_repr_html_',
 '_sc',
 '_schema',
 '_sort_cols',
 '_support_repr_html',
 'agg',
 'alias',
 'approxQuantile',
 'cache',
 'checkpoint',
 'coalesce',
 'colRegex',
 'collect',
 'columns',
 'corr',
 'count',
 'cov',
 'createGlobalTempView',
 'createOrReplaceGlobalTempView',
 'createOrReplaceTempView',
 'createTempView',
 'crossJoin',
 'crosstab',
 'cube',
 'describe',
 'distinct',
 'drop',
 'dropDuplicates',
 'drop_duplicates',
 'dropna',
 'dtypes',
 'exceptAll',
 'explain',
 'fillna',
 'filter',
 'first',
 'foreach',
 'f

In [48]:
ts_df.count()

3261

In [49]:
ts_df.first()['UID']

'16'

In [50]:
out = ts_df.groupBy('Province_State').count().orderBy('Province_State')
out.collect()


[Row(Province_State='Alabama', count=69),
 Row(Province_State='Alaska', count=31),
 Row(Province_State='American Samoa', count=1),
 Row(Province_State='Arizona', count=17),
 Row(Province_State='Arkansas', count=77),
 Row(Province_State='California', count=60),
 Row(Province_State='Colorado', count=66),
 Row(Province_State='Connecticut', count=10),
 Row(Province_State='Delaware', count=5),
 Row(Province_State='Diamond Princess', count=1),
 Row(Province_State='District of Columbia', count=3),
 Row(Province_State='Florida', count=69),
 Row(Province_State='Georgia', count=161),
 Row(Province_State='Grand Princess', count=1),
 Row(Province_State='Guam', count=1),
 Row(Province_State='Hawaii', count=7),
 Row(Province_State='Idaho', count=46),
 Row(Province_State='Illinois', count=104),
 Row(Province_State='Indiana', count=94),
 Row(Province_State='Iowa', count=101),
 Row(Province_State='Kansas', count=107),
 Row(Province_State='Kentucky', count=122),
 Row(Province_State='Louisiana', count=66

In [51]:
ts_df.first()['Province_State']

'American Samoa'

In [52]:
ts_df.dtypes

[('UID', 'string'),
 ('iso2', 'string'),
 ('iso3', 'string'),
 ('code3', 'string'),
 ('FIPS', 'string'),
 ('Admin2', 'string'),
 ('Province_State', 'string'),
 ('Country_Region', 'string'),
 ('Lat', 'string'),
 ('Long_', 'string'),
 ('Combined_Key', 'string'),
 ('1/22/20', 'string'),
 ('1/23/20', 'string'),
 ('1/24/20', 'string'),
 ('1/25/20', 'string'),
 ('1/26/20', 'string'),
 ('1/27/20', 'string'),
 ('1/28/20', 'string'),
 ('1/29/20', 'string'),
 ('1/30/20', 'string'),
 ('1/31/20', 'string'),
 ('2/1/20', 'string'),
 ('2/2/20', 'string'),
 ('2/3/20', 'string'),
 ('2/4/20', 'string'),
 ('2/5/20', 'string'),
 ('2/6/20', 'string'),
 ('2/7/20', 'string'),
 ('2/8/20', 'string'),
 ('2/9/20', 'string'),
 ('2/10/20', 'string'),
 ('2/11/20', 'string'),
 ('2/12/20', 'string'),
 ('2/13/20', 'string'),
 ('2/14/20', 'string'),
 ('2/15/20', 'string'),
 ('2/16/20', 'string'),
 ('2/17/20', 'string'),
 ('2/18/20', 'string'),
 ('2/19/20', 'string'),
 ('2/20/20', 'string'),
 ('2/21/20', 'string'),
 ('2

In [53]:
ts_df.columns

['UID',
 'iso2',
 'iso3',
 'code3',
 'FIPS',
 'Admin2',
 'Province_State',
 'Country_Region',
 'Lat',
 'Long_',
 'Combined_Key',
 '1/22/20',
 '1/23/20',
 '1/24/20',
 '1/25/20',
 '1/26/20',
 '1/27/20',
 '1/28/20',
 '1/29/20',
 '1/30/20',
 '1/31/20',
 '2/1/20',
 '2/2/20',
 '2/3/20',
 '2/4/20',
 '2/5/20',
 '2/6/20',
 '2/7/20',
 '2/8/20',
 '2/9/20',
 '2/10/20',
 '2/11/20',
 '2/12/20',
 '2/13/20',
 '2/14/20',
 '2/15/20',
 '2/16/20',
 '2/17/20',
 '2/18/20',
 '2/19/20',
 '2/20/20',
 '2/21/20',
 '2/22/20',
 '2/23/20',
 '2/24/20',
 '2/25/20',
 '2/26/20',
 '2/27/20',
 '2/28/20',
 '2/29/20',
 '3/1/20',
 '3/2/20',
 '3/3/20',
 '3/4/20',
 '3/5/20',
 '3/6/20',
 '3/7/20',
 '3/8/20',
 '3/9/20',
 '3/10/20',
 '3/11/20',
 '3/12/20',
 '3/13/20',
 '3/14/20',
 '3/15/20',
 '3/16/20',
 '3/17/20',
 '3/18/20',
 '3/19/20',
 '3/20/20',
 '3/21/20',
 '3/22/20',
 '3/23/20',
 '3/24/20',
 '3/25/20',
 '3/26/20',
 '3/27/20',
 '3/28/20',
 '3/29/20',
 '3/30/20',
 '3/31/20',
 '4/1/20',
 '4/2/20',
 '4/3/20',
 '4/4/20',
 '4/5

In [54]:
first_row = ts_df.first()
print(first_row.UID)
print(first_row['UID'])

16
16


In [55]:
ts_df.select('Province_State').show()

+--------------------+
|      Province_State|
+--------------------+
|      American Samoa|
|                Guam|
|Northern Mariana ...|
|         Puerto Rico|
|      Virgin Islands|
|             Alabama|
|             Alabama|
|             Alabama|
|             Alabama|
|             Alabama|
|             Alabama|
|             Alabama|
|             Alabama|
|             Alabama|
|             Alabama|
|             Alabama|
|             Alabama|
|             Alabama|
|             Alabama|
|             Alabama|
+--------------------+
only showing top 20 rows



In [56]:
ts_df.sort('Province_State').select('Province_State').show()

+--------------+
|Province_State|
+--------------+
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
|       Alabama|
+--------------+
only showing top 20 rows



In [57]:

ts_df.filter(ts_df['5/15/20'] != '0').select('Province_State', '5/15/20').orderBy(desc('5/15/20')).show()

+--------------+-------+
|Province_State|5/15/20|
+--------------+-------+
|       Georgia|    999|
|       Florida|    997|
|         Texas|    993|
|       Vermont|     99|
|       Wyoming|     99|
|   Mississippi|     99|
|      Michigan|     99|
|          Ohio|    989|
|   Connecticut|   9881|
|      New York|   9825|
|      Illinois|     98|
|South Carolina|     98|
|       Indiana|    977|
|          Ohio|     97|
| West Virginia|     97|
|      Michigan|     97|
|   Mississippi|     97|
|       Georgia|     97|
|North Carolina|    968|
|         Texas|     96|
+--------------+-------+
only showing top 20 rows



In [58]:
target_date = '6/7/20'
target = ts_df.select('Admin2', 'Province_State', target_date).where(ts_df['Admin2'] != 'null')

In [59]:
target.Admin2

Column<b'Admin2'>

In [60]:
from pyspark.sql.types import (StructField, StringType, StructType, IntegerType)
data_fields = [StructField('Admin2', StringType(), True), StructField('Province_State', StringType(), True),
               StructField(target_date, StringType(), True)]
data_schema = StructType(data_fields)
newDF = spark.createDataFrame(target.rdd, schema=data_schema)
newDF.dtypes

[('Admin2', 'string'), ('Province_State', 'string'), ('6/7/20', 'string')]

In [61]:

target_list = []
for row in target.collect():
    target_list.append(['{}.{}'.format(row['Province_State'], row['Admin2']), int(row[target_date])])
target_list

[['Alabama.Autauga', 265],
 ['Alabama.Baldwin', 313],
 ['Alabama.Barbour', 193],
 ['Alabama.Bibb', 77],
 ['Alabama.Blount', 72],
 ['Alabama.Bullock', 232],
 ['Alabama.Butler', 449],
 ['Alabama.Calhoun', 176],
 ['Alabama.Chambers', 378],
 ['Alabama.Cherokee', 42],
 ['Alabama.Chilton', 110],
 ['Alabama.Choctaw', 155],
 ['Alabama.Clarke', 156],
 ['Alabama.Clay', 29],
 ['Alabama.Cleburne', 18],
 ['Alabama.Coffee', 246],
 ['Alabama.Colbert', 209],
 ['Alabama.Conecuh', 49],
 ['Alabama.Coosa', 40],
 ['Alabama.Covington', 88],
 ['Alabama.Crenshaw', 84],
 ['Alabama.Cullman', 193],
 ['Alabama.Dale', 131],
 ['Alabama.Dallas', 320],
 ['Alabama.DeKalb', 269],
 ['Alabama.Elmore', 411],
 ['Alabama.Escambia', 85],
 ['Alabama.Etowah', 273],
 ['Alabama.Fayette', 18],
 ['Alabama.Franklin', 617],
 ['Alabama.Geneva', 46],
 ['Alabama.Greene', 98],
 ['Alabama.Hale', 202],
 ['Alabama.Henry', 88],
 ['Alabama.Houston', 245],
 ['Alabama.Jackson', 90],
 ['Alabama.Jefferson', 2039],
 ['Alabama.Lamar', 29],
 ['Alab

In [62]:
def get_key(x):
    return x[1]

target_list
sorted(target_list, key = get_key, reverse=True)

# 'New York.New York', 206969 Jun 7
# 'New York.New York', 206511 Jun 6
# 'Georgia.Fulton', 4823 Jun 7
# 'Georgia.Fulton', 4822 Jun 6

[['New York.New York', 206969],
 ['Illinois.Cook', 82427],
 ['California.Los Angeles', 63844],
 ['New York.Nassau', 40904],
 ['New York.Suffolk', 40329],
 ['New York.Westchester', 33954],
 ['Pennsylvania.Philadelphia', 23529],
 ['Massachusetts.Middlesex', 22754],
 ['Michigan.Wayne', 20663],
 ['Florida.Miami-Dade', 19547],
 ['Massachusetts.Suffolk', 19001],
 ['New Jersey.Hudson', 18916],
 ['New Jersey.Bergen', 18512],
 ['New Jersey.Essex', 18077],
 ['New Jersey.Passaic', 16449],
 ["Maryland.Prince George's", 16434],
 ['New Jersey.Middlesex', 16198],
 ['New Jersey.Union', 16186],
 ['Connecticut.Fairfield', 16056],
 ['Massachusetts.Essex', 15217],
 ['Texas.Harris', 14586],
 ['Arizona.Maricopa', 13498],
 ['New York.Rockland', 13325],
 ['Maryland.Montgomery', 12734],
 ['Texas.Dallas', 12093],
 ['Virginia.Fairfax', 12056],
 ['Connecticut.New Haven', 11828],
 ['Massachusetts.Worcester', 11732],
 ['Rhode Island.Providence', 11052],
 ['Connecticut.Hartford', 10809],
 ['New York.Orange', 10514],

### End DF demonstration

In [ ]:
country_region = df.groupBy('Country_Region').count().orderBy(desc('count'))

In [ ]:
print(country_region)

In [ ]:
def count_by_country_region (day):
    df = spark.read.option("header", "true").csv(day)
    return df.groupBy('Country_region').count().orderBy(desc('count'))

In [ ]:

count_by_country_region(day).show()

In [ ]:
import subprocess

dir_in = "/user/student/covid19/daily"
args = "hdfs dfs -ls "+dir_in+" | awk '{print $8}'"
proc = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

s_output, s_err = proc.communicate()
all_dart_dirs = s_output.split()